<a href="https://colab.research.google.com/github/victoralmeida428/Data-Science/blob/main/Corretor%20Machine%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Identificar se a qtd de letras do banco de dados é o suficiente*

In [8]:
with open("artigos.txt", "r") as f:
    artigos = f.read()



In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
palavras_separadas = nltk.tokenize.word_tokenize(artigos)

In [11]:
def separa_palavras(lista_tokens):
  lista_de_palavras = []
  for token in lista_tokens:
    if token.isalpha():
      lista_de_palavras.append(token)
  return lista_de_palavras

In [12]:
lista_palavras = separa_palavras(palavras_separadas)
print(f'O número de palavras é {len(lista_palavras)}')

O número de palavras é 403031


In [13]:
def normalizacao (lista_palavra):
  lista_normalizada = []
  for token in lista_palavra:
    lista_normalizada.append(token.lower())
  return lista_normalizada
lista_normalizada = normalizacao(lista_palavras)
print(lista_normalizada[:6])
print(f'O número de palavras é {len(set(lista_normalizada))}')

['imagem', 'temos', 'a', 'seguinte', 'classe', 'que']
O número de palavras é 18464


# *Função do Corretor por Adição de Letra*

In [14]:
def inserir_letras(fatias):
  novas_palavras = []
  letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
  for E, D in fatias:
    for letra in letras:
      novas_palavras.append(E + letra + D)
  return novas_palavras

In [36]:
def gerador_de_palavras(palavra):
  fatias = []
  for i in range(len(palavra)+1):
    fatias.append((palavra[:i], palavra[i:]))
  palavra_gerada = inserir_letras(fatias)
  palavra_gerada += del_caracter(fatias)
  palavra_gerada += switch_letra(fatias)
  palavra_gerada += permuta(fatias)
  return(palavra_gerada)


In [16]:
frequencia = nltk.FreqDist(lista_normalizada)
total_de_palavras = len(lista_normalizada)

In [17]:
def probabilidade(palavra_gerada):
  return frequencia[palavra_gerada]/total_de_palavras


In [76]:
def corretor (palavra):
  palavras_geradas = gerador_de_palavras(palavra)
  palavra_correta = max(palavras_geradas, key=probabilidade)
  return palavra_correta

In [19]:
corretor("arvoe")

'arvore'

# *Avaliação do Corretor*

In [46]:
def cria_dados_teste (arquivo):
  lista_palavras_teste = []
  f = open(arquivo, 'r')
  for linha in f:
    correta, errada = linha.split()
    lista_palavras_teste.append((correta, errada))
  f.close()
  return lista_palavras_teste


In [59]:
vocabulario = set(lista_normalizada)
def avaliador (testes, vocabulario):
  num_palavras = len(testes)
  acertou = 0
  desconhecida = 0
  for correta, errada in testes:
    palavra_correta = corretor(errada)
    if palavra_correta == correta:
      acertou += 1
    else:
      desconhecida += (correta not in vocabulario)
  taxa_acerto = acertou/num_palavras
  taxa_desconhecida = desconhecida/num_palavras
  resposta = print(f'Taxa de acerto: {taxa_acerto*100:.2f}% \n'
                   f'Total de palavras: {num_palavras}\n'
                   f'Taxa de Palavras Desconhecidas: {taxa_desconhecida*100:.2f}%')
                    
  return resposta

In [56]:
teste = cria_dados_teste('palavras.txt')

Taxa de acerto: 76.34% 
Total de palavras: 186
Taxa de Palavras Desconhecidas: 6.99%


# *Deletando uma letra*

In [23]:
def del_caracter(fatias):
  novas_palavras = []
  for E, D in fatias:
    novas_palavras.append(E + D[1:])
  return novas_palavras

In [57]:
avaliador(teste, vocabulario)

Taxa de acerto: 76.34% 
Total de palavras: 186
Taxa de Palavras Desconhecidas: 6.99%


# *Troca e Permuta letra*

In [25]:
def switch_letra(fatias):
  novas_palavras = []
  letras = 'abcdefghijklmnopqrstuvwxyzàáâãèéêìíîòóôõùúûç'
  for E, D in fatias:
    for letra in letras:
      novas_palavras.append(E + letra + D[1:])
  return novas_palavras

In [26]:
avaliador(teste)

Taxa de acerto: 76.34% 
Total de palavras: 186


In [42]:
def permuta (fatias):
  novas_palavras = []
  for E, D in fatias:
    if len(D)>1:
      novas_palavras.append(E + D[1] + D[0] + D[2:])
  return novas_palavras

In [58]:
avaliador(teste, vocabulario)

Taxa de acerto: 76.34% 
Total de palavras: 186
Taxa de Palavras Desconhecidas: 6.99%


# *Gerador Turbinado*

In [64]:
def gerador_turbinado (palavras_geradas):
  novas_palavras=[]
  for palavra in palavras_geradas:
    novas_palavras += gerador_de_palavras(palavra)
  return novas_palavras

In [73]:
palavras_turbinadas = gerador_turbinado(gerador_de_palavras("lógiiica"))
"lógica" in palavras_turbinadas

True

In [82]:
def novo_corretor (palavra):
  palavras_geradas = gerador_de_palavras(palavra)
  palavras_turbinadas = gerador_turbinado(gerador_de_palavras(palavra))
  todas_palavras = set(palavras_geradas + palavras_turbinadas)
  candidatos = [palavra]
  for palavra in todas_palavras:
    if palavra in todas_palavras:
      candidatos.append(palavra)
  palavra_correta = max(candidatos, key=probabilidade)
  return palavra_correta

In [87]:
def avaliador_novo (testes, vocabulario):
  num_palavras = len(testes)
  acertou = 0
  desconhecida = 0

  for correta, errada in testes:
    palavra_correta = novo_corretor(errada)
    desconhecida += (correta not in vocabulario)
    if palavra_correta == correta:
      acertou += 1     
  taxa_acerto = acertou/num_palavras
  taxa_desconhecida = desconhecida/num_palavras
  resposta = print(f'Taxa de acerto: {taxa_acerto*100:.2f}% \n'
                   f'Total de palavras: {num_palavras}\n'
                   f'Taxa de Palavras Desconhecidas: {taxa_desconhecida*100:.2f}%')
                    
  return resposta

# *Comparando Corretores*

In [89]:
print('eficiencia novo corretor')
avaliador_novo(teste, vocabulario)
print('eficiencia corretor antigo')
avaliador(teste, vocabulario)

eficiencia novo corretor
Taxa de acerto: 55.38% 
Total de palavras: 186
Taxa de Palavras Desconhecidas: 6.99%
eficiencia corretor antigo
Taxa de acerto: 76.34% 
Total de palavras: 186
Taxa de Palavras Desconhecidas: 6.99%
